# PySpark DataFrames and SQL

[Jian Tao](https://orcid.org/0000-0003-4228-6089), Texas A&M University

May 1, 2021



### 1. Set up the PySpark environment first

In [16]:
# For each Google Colab, we will need to run this cell to ensure that PySpark is installed properly.
!pip install pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Colab").config('spark.ui.port', '4050').getOrCreate()

# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip -o ngrok-stable-linux-amd64.zip
# get_ipython().system_raw('./ngrok http 4050 &')
# !curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(\"\nClick me to launch (give it a minute or two)\n\"); print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

### 2. Create a DataFrame by reading from a CSV/JSON file

`spark.read.csv` can only read from local files, so we will have to download the CSV file from the URL first. We can use `SparkFiles` to do that or use `pandas`. For those CSV files with a header, please make sure to set `header=True` in the argument list for `spark.read.csv`. When the data types of the columns are not known, `inferSchema=True` will do the trick to automatically recognize the data types, but it is not perfect. In our example, `Horsepower` is not correctly recognized.

In [98]:
from pyspark import SparkFiles

csv_url = "https://raw.githubusercontent.com/jtao/AdvancedML/main/data/Auto.csv"
json_url = "https://raw.githubusercontent.com/jtao/dswebinar/master/pyspark/Auto.json"

spark.sparkContext.addFile(csv_url)
spark.sparkContext.addFile(json_url)

## One can create a spark dataframe from pandas dataframe as well.
# import pandas as pd
# df = spark.createDataFrame(pd.read_csv(url))

#df = spark.read.csv(SparkFiles.get("Auto.csv"), header=True, sep=",", inferSchema=False)
df = spark.read.csv(SparkFiles.get("Auto.csv"), header=True, sep=",", inferSchema=True)
df.printSchema()
df.show(5)

root
 |-- mpg: double (nullable = true)
 |-- cylinders: integer (nullable = true)
 |-- displacement: double (nullable = true)
 |-- horsepower: string (nullable = true)
 |-- weight: integer (nullable = true)
 |-- acceleration: double (nullable = true)
 |-- year: integer (nullable = true)
 |-- origin: integer (nullable = true)
 |-- name: string (nullable = true)

+----+---------+------------+----------+------+------------+----+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|year|origin|                name|
+----+---------+------------+----------+------+------------+----+------+--------------------+
|18.0|        8|       307.0|       130|  3504|        12.0|  70|     1|chevrolet chevell...|
|15.0|        8|       350.0|       165|  3693|        11.5|  70|     1|   buick skylark 320|
|18.0|        8|       318.0|       150|  3436|        11.0|  70|     1|  plymouth satellite|
|16.0|        8|       304.0|       150|  3433|        12.0|  70|     1|

In [99]:
df = spark.read.json(SparkFiles.get("Auto.json"), multiLine=True)

In [100]:
df.show(10)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|        acceleration|           cylinders|        displacement|          horsepower|                 mpg|                name|              origin|              weight|                year|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|{12.0, 11.5, 10.0...|{8, 8, 8, 6, 6, 4...|{307.0, 350.0, 38...|{130, 165, 170, 8...|{18.0, 15.0, 15.0...|{chevrolet chevel...|{1, 1, 1, 1, 1, 2...|{3504, 3693, 3563...|{70, 70, 70, 73, ...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+



We can define a schema to help `spark.read.csv` to correctly cast the type of all the columns.

In [106]:
from pyspark.sql.types import *

user_schema = StructType([
                     StructField("mpg", IntegerType(), True),
                     StructField("cylinders", IntegerType(), True),
                     StructField("displacement", IntegerType(), True),
                     StructField("horsepower", IntegerType(), True),
                     StructField("weight", IntegerType(), True),
                     StructField("acceleration", DoubleType(), True),
                     StructField("year", IntegerType(), True),
                     StructField("origin", IntegerType(), True),
                     StructField("name", StringType(), True)
])

df = spark.read.csv(SparkFiles.get("Auto.csv"), header=True, sep=",", schema=user_schema, inferSchema=True)

df.printSchema()
df.show(5)

root
 |-- mpg: integer (nullable = true)
 |-- cylinders: integer (nullable = true)
 |-- displacement: integer (nullable = true)
 |-- horsepower: integer (nullable = true)
 |-- weight: integer (nullable = true)
 |-- acceleration: double (nullable = true)
 |-- year: integer (nullable = true)
 |-- origin: integer (nullable = true)
 |-- name: string (nullable = true)

+---+---------+------------+----------+------+------------+----+------+--------------------+
|mpg|cylinders|displacement|horsepower|weight|acceleration|year|origin|                name|
+---+---------+------------+----------+------+------------+----+------+--------------------+
| 18|        8|         307|       130|  3504|        12.0|  70|     1|chevrolet chevell...|
| 15|        8|         350|       165|  3693|        11.5|  70|     1|   buick skylark 320|
| 18|        8|         318|       150|  3436|        11.0|  70|     1|  plymouth satellite|
| 16|        8|         304|       150|  3433|        12.0|  70|     1|    

### 3. Create a Spark DataFrame with a list

In [52]:
auto_list = [(1, 18, "Chevrolet"), (2, 15, "Buick"), (3, 18, "Plymouth"), (4, 16, "Amc"), (5, 17, "Ford")]

df = spark.createDataFrame(auto_list)
df.printSchema()
df.show(5)

from pyspark.sql.types import StructType, StructField, IntegerType, StringType

users_schema = StructType([
                          StructField("id", IntegerType(), True),
                          StructField("mpg", IntegerType(), True),
                          StructField("name", StringType(), True)])

df = spark.createDataFrame(auto_list, schema=users_schema)
df.printSchema()
df.show(5)

root
 |-- _1: long (nullable = true)
 |-- _2: long (nullable = true)
 |-- _3: string (nullable = true)

+---+---+---------+
| _1| _2|       _3|
+---+---+---------+
|  1| 18|Chevrolet|
|  2| 15|    Buick|
|  3| 18| Plymouth|
|  4| 16|      Amc|
|  5| 17|     Ford|
+---+---+---------+

root
 |-- id: integer (nullable = true)
 |-- mpg: integer (nullable = true)
 |-- name: string (nullable = true)

+---+---+---------+
| id|mpg|     name|
+---+---+---------+
|  1| 18|Chevrolet|
|  2| 15|    Buick|
|  3| 18| Plymouth|
|  4| 16|      Amc|
|  5| 17|     Ford|
+---+---+---------+



### 3. Create a Spark DataFrame with a list of dictionaries

In [53]:
auto_list = [{"id": 1, "mpg": 18, "name": "Chevrolet"}, 
                {"id": 2, "mpg": 15, "name": "Buick"}, 
                {"id": 3, "mpg": 18, "name": "Plymouth"}, 
                {"id": 4, "mpg": 16, "name": "Amc"}, 
                {"id": 5, "mpg": 17, "name": "Ford"}]
df = spark.createDataFrame(auto_list)
df.printSchema()
df.show(5)

root
 |-- id: long (nullable = true)
 |-- mpg: long (nullable = true)
 |-- name: string (nullable = true)

+---+---+---------+
| id|mpg|     name|
+---+---+---------+
|  1| 18|Chevrolet|
|  2| 15|    Buick|
|  3| 18| Plymouth|
|  4| 16|      Amc|
|  5| 17|     Ford|
+---+---+---------+



In [108]:
df=pd.read_csv(url)

In [109]:
df.to_json("Auto.json", orient='records')

In [ ]:
spark.stop()